In [1]:
import pandas as pd 
import numpy as np
import sklearn
import os
import gc
# parallel apply!
import swifter
from tqdm.autonotebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
data_dir = './data'

In [3]:
age_test_df = pd.read_csv(os.path.join(data_dir, "age_test.csv"), header=None)
age_train_df = pd.read_csv(os.path.join(data_dir, "age_train.csv"), header=None)
app_info_df = pd.read_csv(os.path.join(data_dir, "app_info.csv"), header=None)
user_app_actived_df = pd.read_csv(os.path.join(data_dir, "user_app_actived.csv"), header=None)
user_basic_info_df = pd.read_csv(os.path.join(data_dir, "user_basic_info.csv"), header=None)
user_behavior_info_df = pd.read_csv(os.path.join(data_dir, "user_behavior_info.csv"), header=None)
usage_summary_df = pd.read_csv(os.path.join(data_dir, "usages_summary.csv"))
user_app_usage = None

In [4]:
age_train_df.columns = ['uid', 'age_group']
age_test_df.columns = ['uid']
user_basic_info_df.columns = ['uid', 'gender', 'city', 'prodName', 'ramCap', 'ramLeft', 
                            'romCap', 'romLeft', 'color', 'fontSize', 'ct', 'carrier', 'os']
user_behavior_info_df.columns = ['uid', 'bootTimes', 'aTimes', 'bTimes', 'cTimes', 'dTimes', 
                                 'eTimes', 'fTimes', 'fSum']
user_app_actived_df.columns = ['uid', 'appId']
app_info_df.columns = ['appId', 'category']

In [5]:
import sklearn.preprocessing
def encodeCategory(df):
    encoder = sklearn.preprocessing.LabelEncoder()
    return encoder.fit_transform(df)

### process basic info

In [9]:
user_basic_info_df['city'].head()

0     50
1    301
2    201
3    287
4    319
Name: city, dtype: int64

In [8]:
basic_cates = ['city', 'prodName', 'color', 'carrier']
for field in basic_cates:
    user_basic_info_df[field] = encodeCategory(user_basic_info_df[field].astype(str))

In [7]:
# [4g,3g,2g,wifi]
ct_onehot_dict = {
    "4g": [1,0,0,0],
    'wifi': [0,0,0,1],
    '4g#wifi': [1,0,0,1],
    0: [0,0,0,0], #nan
    '3g#wifi': [0,1,0,1],
    '3g': [0,1,0,0],
    '2g#wifi': [0,0,1,1],
    '2g': [0,0,1,0]
}
user_basic_info_df['4g'], user_basic_info_df['3g'], user_basic_info_df['2g'], user_basic_info_df['wifi'] = zip(*user_basic_info_df['ct'].fillna(0).apply(lambda c: ct_onehot_dict[c]))
user_basic_info_df.drop('ct', axis=1, inplace=True)

### process user_app_actived and app_info

In [10]:
app_info_df['category_id'] = encodeCategory(app_info_df['category'])
app_info_map = {r: [] for r in tqdm(app_info_df['appId'].unique(), leave=False)}

for _, r in tqdm(app_info_df.iterrows(), total=app_info_df.shape[0], leave=False):
    app_info_map[r['appId']].append(r['category'])


In [11]:
user_app_actived_df['app_category'] = user_app_actived_df['appId'].progress_apply(
    lambda a: ' '.join([' '.join(app_info_map.get(appId, ['unknown'])) for appId in a.split('#')]))

In [12]:
import sklearn.feature_extraction.text
# vectorizer = sklearn.feature_extraction.text.TfidfVectorizer()
w2v_vectorizer = sklearn.feature_extraction.text.CountVectorizer()
w2v_app_vectors = w2v_vectorizer.fit_transform(user_app_actived_df['app_category'])
tf_idf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer()
tf_idf_vectors = tf_idf_vectorizer.fit_transform(user_app_actived_df['app_category'])

In [13]:
user_app_vec = pd.DataFrame(user_app_actived_df['uid'])
for i in tqdm(range(len(w2v_vectorizer.vocabulary_)), leave=False):
    user_app_vec['app_w2v_%s'%list(w2v_vectorizer.vocabulary_.keys())[i]] = w2v_app_vectors[:, i].toarray()
for i in tqdm(range(len(tf_idf_vectorizer.vocabulary_)), leave=False):
    user_app_vec['app_tfidf_%s'%list(tf_idf_vectorizer.vocabulary_.keys())[i]] = tf_idf_vectors[:, i].toarray()
user_app_vec['total_app'] = user_app_actived_df['appId'].progress_apply(
    lambda a: len(a.split('#')))

In [12]:
# all_apps = set()
# for apps in tqdm(user_app_actived_df['appId'], leave=False):
#     all_apps.update(apps.split('#'))
# app_indices = {k: i for i, k in tqdm(enumerate(all_apps), leave=False)}
# user_app_vec['app_series'] = user_app_actived_df['appId'].progress_apply(
#     lambda a: ' '.join([str(app_indices[app]) for app in (a if a else '').split('#')]))

## Prepare training features

In [12]:
user_info_df = user_basic_info_df.merge(user_behavior_info_df, on='uid')

In [13]:
features_df = user_info_df.merge(user_app_vec, on='uid', how='left') 

In [14]:
features_df = features_df.merge(usage_summary_df, on='uid', how='left') 

In [15]:
features_df.sort_values(by='uid', inplace=True)
features_df.reset_index(drop=True, inplace=True)

In [16]:
features_df.columns, len(features_df.columns)

(Index(['uid', 'gender', 'city', 'prodName', 'ramCap', 'ramLeft', 'romCap',
        'romLeft', 'color', 'fontSize',
        ...
        'duration_网络游戏', 'usage_益智棋牌', 'times_益智棋牌', 'duration_益智棋牌',
        'usage_儿童', 'times_儿童', 'duration_儿童', 'usage_出行导航', 'times_出行导航',
        'duration_出行导航'],
       dtype='object', length=212), 212)

In [17]:
categorical_features = ['gender', 'city', 'prodName', 'color', 'carrier']

In [18]:
# def one_hot_category(df, feature_name):
#     dum = pd.get_dummies(df[feature_name])
#     for col in dum.columns:
#         df['%s_%s'%(feature_name, col)] = dum[col]
#     df.drop(feature_name, axis=1, inplace=True)
# for cate in categorical_features:
#     one_hot_category(features_df, cate)

In [19]:
del user_basic_info_df
del user_behavior_info_df
del user_app_vec
del user_info_df
del usage_summary_df
gc.collect()

97

In [20]:
train_data = features_df.merge(age_train_df, on='uid', how='right')
test_x = features_df.merge(age_test_df, on='uid', how='right')

In [21]:
del features_df
gc.collect()

68

In [22]:
train_x = train_data.drop(['uid', 'age_group'], axis=1)
train_y = train_data['age_group']

In [23]:
del train_data
gc.collect()

47

In [24]:
# train_x.fillna(train_x.mean(), inplace=True)
# test_x.fillna(train_x.mean(), inplace=True)

In [25]:
train_x = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)
test_x = pd.DataFrame(test_x)

In [26]:
train_dir = './train-data'
train_x.to_csv(os.path.join(train_dir, 'train_x.csv'), index=False, header=True)
train_y.to_csv(os.path.join(train_dir, 'train_y.csv'), index=False, header=True)
test_x.to_csv(os.path.join(train_dir, 'test_x.csv'), index=False, header=True)